# Full Project Code

The first part of this notebook is responsible for transforming the raw json data into a usable tabular format. Due to the robustness of XGBoost as an algorithm, we do not need to handle missing values, and can pass them directly as nulls.

Note that we must have created the relevant csv files before running this code, which can be done using `scripts/convert_json_data.py`.

## Setup
This part of the notebook imports all dependencies, and sets up any global constants we will need.

In [2]:
import os
from typing import Union

import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import xgboost as xg
from scipy.interpolate import interp1d
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.preprocessing import LabelEncoder
from statsmodels.nonparametric.smoothers_lowess import lowess

In [3]:
os.chdir(os.path.join("/", "home", "walkerdavis", "projects", "mpsc"))
DATA_ROOT = os.path.join("data", "processed")

np.random.seed(42)

## Data Processing

The `GrowthDataTransformation` class encapsulates all data loading and feature engineering for the model.

### `__init__(self)`
Initializes the class instance. Sets up placeholders for the consolidated output dataframe (`self.consolidated_data`) and a dictionary for patient-specific growth model interpolators (`self.growth_models`).

### `load_and_process_demographics(self, demographics_path: str)`
Reads the demographics CSV and cleans the resulting dataframe:
- Converts Gestational Age from a string to numeric, and extracts the integer.
- Parses DOB, and Data and Fluid Start and End as datetime objects.
- Filters out records with future fluid dates, missing/invalid GA, or birth weight outliers.

### `load_and_process_weight_data(self, weight_path: str):`
Loads and cleans weight measurement data.
- Converts weights to grams (assumes original is in ounces).
- Adds a column marking these as weight measurements (`OBX`).
- Converts timestamps to datetime.
- Removes rows with missing weight values.

### `create_growth_tibble(self, demographics, weight_data)`:
Joins demographics and weight data, and computes time-relative variables.
- Merges demographics and weight by `ID`.
- Calculates days since birth for each weight entry.
- Computes post-menstrual age (PMA) in days and weeks.

### `fit_growth_models(self, growth_data, frac=0.3)`:
Fits a smoothed weight curve for each patient and saves interpolation functions to the main class.
- Filters patients with at least 3 weight observations.
- For each, sorts by age, fits a LOESS smoothed curve to weight over time. NOTE: I had attemtped to use an alternative smoothing algorithm that did not need to reference future values, however I ran into issue where the algorithm would generate massive amounts of missing values later in the dataset.
- Builds an interpolating function for each patient’s smoothed weights.
- Stores models and returns all patient data with smoothed weights included.

### `load_and_process_energy_data(self, energy_path: str)`:
Loads and processes daily energy intake data.
- Reads CSV, converts timestamps.
- Marks whether energy was delivered parenterally (IV) or enterally.
- Aggregates total energy per day and route.
- Pivots into one row per patient per day, with enteral, parenteral, and total energy columns.

### `load_and_process_pulse_data(self, pulse_path)`:
Loads and processes pulse rate data.
- Reads CSV, extracts date.
- Aggregates mean daily pulse for each patient.

### `load_and_process_spo2_data(self, spo2_path)`:
Loads and processes daily oxygen saturation (SpO2) data.
- Reads CSV, extracts date.
- Aggregates mean daily SpO2 for each patient.

### `load_and_process_stool_weight_data(self, stool_path)`:
Loads and sums daily stool output.
- Reads CSV, extracts date.
- Aggregates total stool output per patient per day.

### `load_and_process_bp_data(self, bp_path: str)`:
Loads and processes blood pressure readings.
- Reads CSV, parses dates.
- Splits BP readings into systolic and diastolic values.
- Aggregates mean daily systolic/diastolic BP per patient.

### `load_and_process_resp_data(self, resp_path)`:
Loads and processes daily respiratory rate data.
- Reads CSV, extracts date.
- Aggregates mean daily respiratory rate per patient.

### `load_and_process_io_data(self, io_path)`:
Loads and processes daily input/output data.
- Reads CSV, parses date.
- Pivots to one row per patient per day, with each OBX variable as a column.

### `predict_smoothed_weight(self, patient_id, day_from_birth)`:
For a given patient and day, returns the smoothed weight value using their fitted curve.
- Looks up the interpolation model and evaluates at the requested day.

### `calculate_weight_derivatives(self, patient_id, day_from_birth)`:
Calculates the patient’s weight velocity (gain) on a specific day.
- Uses smoothed weights to estimate daily and weekly gains via finite differences.

### `calculate_rolling_statistics(self, df, patient_col, date_col, value_col, window=7)`:
Adds rolling mean and standard deviation for a feature, per patient.
- Sorts and groups by patient, computes rolling window stats for a given column.

### `consolidate_all_data(self, ...)`:
Merges all available data into a single, comprehensive table and computes derived features.
- Joins all processed daily data on patient and date.
- Computes key clinical time variables (days since birth, PMA).
- Encodes sex.
- Fills in smoothed weights and weight gains.
- Calculates per-kg rates for energy, fluid, and other features.
- Adds rolling stats and a categorical growth stage.
- Filters to relevant PMA window.
- Randomly splits patients into train/test groups.

### `get_normal_weekly_growth(self, ga, week_from_birth, weight_g)`:
Returns the expected (“normal”) weekly weight gain for a given gestational age and week.
- Based on clinical conventions, uses fixed or proportional values by age.

### `add_normal_growth_comparison(self)`:
Appends a column for expected normal weekly growth to the consolidated data.
- For each row, computes the expected growth using get_normal_weekly_growth.

### `create_feature_list(self)`:
Assemble a full feature list
- Lists key base features (per-kg, etc.).
- Optionally includes fluid intake/output and their rolling stats, if the base features are present.
- Extends list to include squared and square root transformations.

### `create_polynomial_features(self)`:
Adds polynomial and root-transformed versions of all base features.
- For each feature, adds a squared and square root column (if non-negative).

In [4]:
class GrowthDataTransformation:
    def __init__(self):
        self.consolidated_data = None
        self.growth_models = {}

    def load_and_process_demographics(self, demographics_path: str) -> pd.DataFrame:
        """Load and process demographics data"""
        demographics = pd.read_csv(demographics_path)

        demographics["GA"] = pd.to_numeric(
            demographics["GA"].str.extract(r"(\d+)")[0], errors="coerce"
        )

        demographics["DOB"] = pd.to_datetime(demographics["DOB"])
        demographics["DataStart"] = pd.to_datetime(demographics["DataStart"])
        demographics["DataEnd"] = pd.to_datetime(demographics["DataEnd"])
        demographics["FluidStart"] = pd.to_datetime(demographics["FluidStart"])
        demographics["FluidEnd"] = pd.to_datetime(demographics["FluidEnd"])

        demographics = demographics[
            (demographics["FluidStart"] < pd.to_datetime("2200-01-01"))
            & (demographics["GA"].notna())
            & (demographics["GA"] > 20)
            & (demographics["BW"].notna())
            & (demographics["BW"] > 200)
        ]

        return demographics

    def load_and_process_weight_data(self, weight_path: str) -> pd.DataFrame:
        """Load and process weight data"""
        weight_data = pd.read_csv(weight_path)
        assert isinstance(weight_data, pd.DataFrame)

        weight_data["Value"] = weight_data["Value"] * 28.35
        weight_data["OBX"] = "WeightG"
        weight_data["DateTime"] = pd.to_datetime(weight_data["DateTime"])

        weight_data = weight_data.dropna(subset=["Value"])

        return weight_data[["ID", "DateTime", "OBX", "Value"]]

    def create_growth_tibble(
        self, demographics: pd.DataFrame, weight_data: pd.DataFrame
    ) -> pd.DataFrame:
        """Join demographics and weight data: compute additional variables"""
        growth_data = demographics[["ID", "DOB", "GA", "BW", "Sex"]].merge(
            weight_data, on="ID", how="inner"
        )

        growth_data["DayFromBirthNumeric"] = (
            growth_data["DateTime"] - growth_data["DOB"]
        ).dt.days.astype(float)

        growth_data["PMADays"] = (
            7 * growth_data["GA"] + growth_data["DayFromBirthNumeric"]
        ).astype(int)

        growth_data["PMAWeeks"] = (growth_data["PMADays"] / 7).astype(int)

        return growth_data

    def fit_growth_models(self, growth_data: pd.DataFrame, frac: float = 0.3):
        patient_counts = growth_data.groupby("ID").size()
        valid_patients = patient_counts[patient_counts >= 3].index

        growth_data = growth_data[growth_data["ID"].isin(valid_patients)]

        models = {}
        smoothed_data = []

        print(f"Num Valid Patients: {len(valid_patients)}")

        for patient_id in valid_patients:
            patient_data = growth_data[growth_data["ID"] == patient_id].copy()            
            patient_data = patient_data.sort_values("DayFromBirthNumeric")

            x = patient_data["DayFromBirthNumeric"].values
            y = patient_data["Value"].values

            if len(np.unique(x)) < 3:
                continue

            try:
                """
                This line currently uses past values, which introduces data leakage into
                any predictor that uses the smoothed weight. Use a simple moving
                average instead.
                """
                fitted = lowess(y, x, frac=frac, return_sorted=False)

                # fitted = pd.Series(y).expanding(min_periods=1).mean().values

                interp_func = interp1d(
                    x,
                    fitted,
                    kind="linear",
                    bounds_error=False,
                    fill_value="extrapolate",
                )

                models[patient_id] = interp_func

                patient_data["fitted"] = interp_func(x)
                smoothed_data.append(patient_data)

            except Exception as e:
                print(f"Error fitting LOESS for patient {patient_id}: {e}")
                continue

        self.growth_models = models

        smoothed_data = pd.concat(smoothed_data, axis=0).reset_index(drop=True)

        return smoothed_data

    def load_and_process_energy_data(self, energy_path: str) -> pd.DataFrame:
        energy_data = pd.read_csv(energy_path)
        energy_data["DateTime"] = pd.to_datetime(energy_data["DateTime"])
        energy_data["StartDate"] = energy_data["DateTime"].dt.date

        energy_data["Parenteral"] = energy_data["TreatmentRoute"].isin(["IV"])

        daily_energy = (
            energy_data.groupby(["ID", "StartDate", "Parenteral"])["Value"]
            .sum()
            .reset_index()
        )
        daily_energy.columns = ["ID", "StartDate", "Parenteral", "DailyEnergy"]

        daily_energy_pivot = daily_energy.pivot_table(
            index=["ID", "StartDate"],
            columns="Parenteral",
            values="DailyEnergy",
            fill_value=0,
        ).reset_index()

        daily_energy_pivot.columns = [
            "ID",
            "StartDate",
            "DailyEnergyEnteral",
            "DailyEnergyParenteral",
        ]
        daily_energy_pivot["DailyEnergy"] = (
            daily_energy_pivot["DailyEnergyEnteral"]
            + daily_energy_pivot["DailyEnergyParenteral"]
        )

        return daily_energy_pivot

    def load_and_process_pulse_data(self, pulse_path):
        pulse_data = pd.read_csv(pulse_path)
        pulse_data["DateTime"] = pd.to_datetime(pulse_data["DateTime"])
        pulse_data["StartDate"] = pulse_data["DateTime"].dt.date

        daily_pulse = (
            pulse_data.groupby(["ID", "StartDate"])["Value"].mean().reset_index()
        )
        daily_pulse.columns = ["ID", "StartDate", "MeanDailyPulse"]

        return daily_pulse

    def load_and_process_spo2_data(self, spo2_path):
        spo2_data = pd.read_csv(spo2_path)
        spo2_data["DateTime"] = pd.to_datetime(spo2_data["DateTime"])
        spo2_data["StartDate"] = spo2_data["DateTime"].dt.date

        daily_spo2 = (
            spo2_data.groupby(["ID", "StartDate"])["Value"].mean().reset_index()
        )
        daily_spo2.columns = ["ID", "StartDate", "MeanDailySPO2"]

        return daily_spo2

    def load_and_process_stool_weight_data(self, stool_path):
        stool_data = pd.read_csv(stool_path)
        stool_data["DateTime"] = pd.to_datetime(stool_data["DateTime"])
        stool_data["StartDate"] = stool_data["DateTime"].dt.date

        daily_sum_stool_weight = (
            stool_data.groupby(["ID", "StartDate"])["Value"].sum().reset_index()
        )
        daily_sum_stool_weight.columns = ["ID", "StartDate", "SumDailyStoolWeight"]

        return daily_sum_stool_weight

    def load_and_process_bp_data(self, bp_path: str) -> pd.DataFrame:
        bp_data = pd.read_csv(bp_path)
        bp_data["DateTime"] = pd.to_datetime(bp_data["DateTime"])
        bp_data["StartDate"] = bp_data["DateTime"].dt.date

        bp_data[["systolic_pb", "diastolic_bp"]] = (
            bp_data["Value"].str.split("/", expand=True).astype(float)
        )

        daily_bp = (
            bp_data.groupby(["ID", "StartDate"])[["systolic_pb", "diastolic_bp"]]
            .mean()
            .reset_index()
        )
        daily_bp.columns = [
            "ID",
            "StartDate",
            "MeanDailySystolicBP",
            "MeanDailyDiastolicBP",
        ]

        return daily_bp

    def load_and_process_resp_data(self, resp_path):
        resp_data = pd.read_csv(resp_path)
        resp_data["DateTime"] = pd.to_datetime(resp_data["DateTime"])
        resp_data["StartDate"] = resp_data["DateTime"].dt.date

        daily_resp = (
            resp_data.groupby(["ID", "StartDate"])["Value"].mean().reset_index()
        )
        daily_resp.columns = ["ID", "StartDate", "MeanDailyResp"]

        return daily_resp

    def load_and_process_io_data(self, io_path):
        io_data = pd.read_csv(io_path)
        io_data["StartDate"] = pd.to_datetime(io_data["StartDate"]).dt.date

        io_pivot = io_data.pivot_table(
            index=["ID", "StartDate"], columns="OBX", values="Value", fill_value=0
        ).reset_index()

        return io_pivot

    def predict_smoothed_weight(self, patient_id, day_from_birth):
        if patient_id in self.growth_models:
            return self.growth_models[patient_id](day_from_birth)
        return np.nan

    def calculate_weight_derivatives(self, patient_id, day_from_birth):
        if patient_id not in self.growth_models:
            return np.nan, np.nan

        model = self.growth_models[patient_id]

        try:
            current_weight = model(day_from_birth)
            next_day_weight = model(day_from_birth + 1)
            daily_gain = next_day_weight - current_weight

        except Exception:
            daily_gain = np.nan

        try:
            week_later_weight = model(day_from_birth + 1)
            week_earlier_weight = model(day_from_birth - 6)
            weekly_gain = (week_later_weight - week_earlier_weight) / 7
        except Exception:
            weekly_gain = np.nan

        return daily_gain, weekly_gain

    def calculate_rolling_statistics(
        self, df, patient_col, date_col, value_col, window=7
    ):
        df = df.sort_values([patient_col, date_col])

        df[f"{value_col}_rolling_mean"] = df.groupby(patient_col)[value_col].transform(
            lambda x: x.rolling(window=window, min_periods=1).mean()
        )
        df[f"{value_col}_rolling_std"] = df.groupby(patient_col)[value_col].transform(
            lambda x: x.rolling(window=window, min_periods=1).std()
        )

        return df

    def consolidate_all_data(
        self,
        demographics,
        energy_data,
        pulse_data,
        io_data,
        spo2_data,
        resp_data,
        stool_data,
        bp_data,
    ):
        consolidated = demographics[["ID", "DOB", "GA", "BW", "Sex"]].copy()
        consolidated = consolidated.merge(energy_data, on="ID", how="inner")
        consolidated = consolidated.merge(
            pulse_data, on=["ID", "StartDate"], how="inner"
        )
        consolidated = consolidated.merge(io_data, on=["ID", "StartDate"], how="inner")
        consolidated = consolidated.merge(
            spo2_data, on=["ID", "StartDate"], how="inner"
        )
        consolidated = consolidated.merge(
            resp_data, on=["ID", "StartDate"], how="inner"
        )
        consolidated = consolidated.merge(
            stool_data, on=["ID", "StartDate"], how="inner"
        )
        consolidated = consolidated.merge(bp_data, on=["ID", "StartDate"], how="inner")

        consolidated["DayFromBirthNumeric"] = (
            pd.to_datetime(consolidated["StartDate"]) - consolidated["DOB"]
        ).dt.days.astype(float)

        consolidated["PMADays"] = (
            7 * consolidated["GA"] + consolidated["DayFromBirthNumeric"]
        ).astype(int)

        consolidated["SexMInt"] = (consolidated["Sex"] == "M").astype(int)

        consolidated["SmoothedWeightG"] = consolidated.apply(
            lambda row: self.predict_smoothed_weight(
                row["ID"], row["DayFromBirthNumeric"]
            ),
            axis=1,
        )

        weight_gains = consolidated.apply(
            lambda row: self.calculate_weight_derivatives(
                row["ID"], row["DayFromBirthNumeric"]
            ),
            axis=1,
        )

        consolidated["DailyWeightGainGPerDay"] = [x[0] for x in weight_gains]
        consolidated["WeeklyWeightGainGPerDay"] = [x[1] for x in weight_gains]

        consolidated["DailyEnergyPerKG"] = (
            consolidated["DailyEnergy"] * 1000 / consolidated["SmoothedWeightG"]
        )
        consolidated["DailyEnergyParenteralPerKG"] = (
            consolidated["DailyEnergyParenteral"]
            * 1000
            / consolidated["SmoothedWeightG"]
        )
        consolidated["DailyWeightGainGPerKGPerDay"] = (
            consolidated["DailyWeightGainGPerDay"]
            * 1000
            / consolidated["SmoothedWeightG"]
        )
        consolidated["WeeklyWeightGainGPerKGPerDay"] = (
            consolidated["WeeklyWeightGainGPerDay"]
            * 1000
            / consolidated["SmoothedWeightG"]
        )
        consolidated["MeanDailyPulsePerKG"] = (
            consolidated["MeanDailyPulse"] * 1000 / consolidated["SmoothedWeightG"]
        )

        if "DailyFluidIntake" in consolidated.columns:
            consolidated["DailyFluidIntakePerKG"] = (
                consolidated["DailyFluidIntake"]
                * 1000
                / consolidated["SmoothedWeightG"]
            )
        if "DailyFluidOutput" in consolidated.columns:
            consolidated["DailyFluidOutputPerKG"] = (
                consolidated["DailyFluidOutput"]
                * 1000
                / consolidated["SmoothedWeightG"]
            )

        if "DailyFluidIntake" in consolidated.columns:
            consolidated = self.calculate_rolling_statistics(
                consolidated, "ID", "StartDate", "DailyFluidIntake"
            )
        if "DailyFluidOutput" in consolidated.columns:
            consolidated = self.calculate_rolling_statistics(
                consolidated, "ID", "StartDate", "DailyFluidOutput"
            )

        consolidated["GrowthStage"] = np.where(
            consolidated["PMADays"] < 184,
            1,
            np.where(consolidated["PMADays"] < 237, 2, 3),
        )

        consolidated = consolidated[consolidated["PMADays"] < 280]

        patient_ids = consolidated["ID"].unique()
        train_ids = np.random.choice(
            patient_ids, size=int(len(patient_ids) * 0.5), replace=False
        )
        consolidated["Train"] = consolidated["ID"].isin(train_ids)

        self.consolidated_data = consolidated
        return consolidated

    def get_normal_weekly_growth(self, ga, week_from_birth, weight_g):
        pma = ga + week_from_birth - 1

        if week_from_birth == 1:
            return 0
        elif pma > 34:
            return 17 * weight_g / 1000
        else:
            return 30

    def add_normal_growth_comparison(self):
        if self.consolidated_data is None:
            raise ValueError("No consolidated data available.")

        self.consolidated_data["WeekFromBirth"] = (
            self.consolidated_data["DayFromBirthNumeric"] / 7
        ).astype(int) + 1

        self.consolidated_data["NormalWeeklyGrowthGPerDay"] = (
            self.consolidated_data.apply(
                lambda row: self.get_normal_weekly_growth(
                    row["GA"], row["WeekFromBirth"], row["SmoothedWeightG"]
                ),
                axis=1,
            )
        )

        return self.consolidated_data

    def create_feature_list(self):
        base_features = [
            "GA",
            "BW",
            "DailyEnergy",
            "DailyEnergyParenteral",
            "DayFromBirthNumeric",
            "SmoothedWeightG",
            "MeanDailyPulse",
            "PMADays",
            "SexMInt",
            "DailyEnergyPerKG",
            "DailyEnergyParenteralPerKG",
            "MeanDailyPulsePerKG",
            "MeanDailySystolicBP",
            "MeanDailyDiastolicBP",
            "SumDailyStoolWeight",
            "MeanDailyResp",
            "MeanDailySPO2",
        ]

        if "DailyFluidIntake" in self.consolidated_data.columns:
            base_features.extend(
                [
                    "DailyFluidIntake",
                    "DailyFluidIntakePerKG",
                    "DailyFluidIntake_rolling_mean",
                    "DailyFluidIntake_rolling_std",
                ]
            )

        if "DailyFluidOutput" in self.consolidated_data.columns:
            base_features.extend(
                [
                    "DailyFluidOutput",
                    "DailyFluidOutputPerKG",
                    "DailyFluidOutput_rolling_mean",
                    "DailyFluidOutput_rolling_std",
                ]
            )

        extended_features = base_features.copy()
        for feature in base_features:
            if feature in self.consolidated_data.columns:
                extended_features.append(f"{feature}_squared")
                extended_features.append(f"{feature}_sqrt")

        return extended_features

    def create_polynomial_features(self):
        if self.consolidated_data is None:
            raise ValueError("No consolidated data available.")

        base_features = self.create_feature_list()

        for feature in base_features:
            if feature in self.consolidated_data.columns:
                self.consolidated_data[f"{feature}_squared"] = (
                    self.consolidated_data[feature] ** 2
                )

                self.consolidated_data[f"{feature}_sqrt"] = np.where(
                    self.consolidated_data[feature] >= 0,
                    self.consolidated_data[feature] ** 0.5,
                    np.nan,
                )

        return self.consolidated_data


This section of code is simply responsible for calling all of the top level functions found in the `GrowthDataTransformation()` class. The final consolidated csv file is built and saved here. Note that if the consolidated csv is already created, this data creation code blocks do not need to be run again.

In [5]:
analysis = GrowthDataTransformation()

demographics = analysis.load_and_process_demographics(
    os.path.join(DATA_ROOT, "Demographics.csv")
)
weight_data = analysis.load_and_process_weight_data(
    os.path.join(DATA_ROOT, "WeightObservations.csv")
)

growth_data = analysis.create_growth_tibble(demographics, weight_data)
smoothed_growth = analysis.fit_growth_models(growth_data)

energy_data = analysis.load_and_process_energy_data(
    os.path.join(DATA_ROOT, "CalculatedEnergyObservations.csv")
)

pulse_data = analysis.load_and_process_pulse_data(
    os.path.join(DATA_ROOT, "PulseObservations.csv")
)

spo2_data = analysis.load_and_process_spo2_data(os.path.join(DATA_ROOT, "SPO2Obs.csv"))

resp_data = analysis.load_and_process_resp_data(os.path.join(DATA_ROOT, "RespObs.csv"))


io_data = analysis.load_and_process_io_data(os.path.join(DATA_ROOT, "DailyIO.csv"))

stool_data = analysis.load_and_process_stool_weight_data(
    os.path.join(DATA_ROOT, "StoolWeightObservations.csv")
)

bp_data = analysis.load_and_process_bp_data(
    os.path.join(DATA_ROOT, "BPObservations.csv")
)

consolidated = analysis.consolidate_all_data(
    demographics,
    energy_data,
    pulse_data,
    io_data,
    spo2_data,
    resp_data,
    stool_data,
    bp_data,
)

Num Valid Patients: 835


/home/walkerdavis/projects/mpsc/.venv/lib/python3.12/site-packages/statsmodels/nonparametric/smoothers_lowess.py:226: RuntimeWarning: invalid value encountered in divide
  res, _ = _lowess(y, x, x, np.ones_like(x),
/home/walkerdavis/projects/mpsc/.venv/lib/python3.12/site-packages/scipy/interpolate/_interpolate.py:497: RuntimeWarning: invalid value encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
/home/walkerdavis/projects/mpsc/.venv/lib/python3.12/site-packages/scipy/interpolate/_interpolate.py:497: RuntimeWarning: invalid value encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
/home/walkerdavis/projects/mpsc/.venv/lib/python3.12/site-packages/scipy/interpolate/_interpolate.py:497: RuntimeWarning: invalid value encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
/home/walkerdavis/projects/mpsc/.venv/lib/python3.12/site-packages/scipy/interpolate/_interpolate.py:497: RuntimeWarning: invalid value encountered in divide
  slop

In [6]:
final_data = analysis.add_normal_growth_comparison()
final_data = analysis.create_polynomial_features()

final_data.to_csv(os.path.join(DATA_ROOT, "Consolidated.csv"), index=False)

final_data

/tmp/ipykernel_253282/3273377643.py:476: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.consolidated_data[f"{feature}_squared"] = (
/tmp/ipykernel_253282/3273377643.py:480: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.consolidated_data[f"{feature}_sqrt"] = np.where(
/tmp/ipykernel_253282/3273377643.py:476: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.

,ID,DOB,GA,BW,Sex,StartDate,DailyEnergyEnteral,DailyEnergyParenteral,DailyEnergy,MeanDailyPulse,...,DailyFluidOutputPerKG_sqrt_squared,DailyFluidOutputPerKG_sqrt_sqrt,DailyFluidOutput_rolling_mean_squared_squared,DailyFluidOutput_rolling_mean_squared_sqrt,DailyFluidOutput_rolling_mean_sqrt_squared,DailyFluidOutput_rolling_mean_sqrt_sqrt,DailyFluidOutput_rolling_std_squared_squared,DailyFluidOutput_rolling_std_squared_sqrt,DailyFluidOutput_rolling_std_sqrt_squared,DailyFluidOutput_rolling_std_sqrt_sqrt
25258,Lurie_000008,2023-01-01,38.0,2620,M,2023-01-07,43.94,172.59,216.53,121.400000,...,41.982975,2.545472,3.224179e+08,134.000000,134.000000,3.402328,NaN,NaN,NaN,NaN
25259,Lurie_000008,2023-01-01,38.0,2620,M,2023-01-08,27.04,290.75,317.79,106.619048,...,140.580093,3.443348,6.975757e+09,289.000000,289.000000,4.123106,2.308802e+09,219.203102,219.203102,3.847793
25260,Lurie_000008,2023-01-01,38.0,2620,M,2023-01-09,54.08,288.30,342.38,126.666667,...,166.960508,3.594624,1.807531e+10,366.666667,366.666667,4.375905,1.774207e+09,205.234825,205.234825,3.784973
25261,Lurie_000008,2023-01-01,38.0,2620,M,2023-01-10,47.32,302.52,349.84,150.400000,...,87.341755,3.057071,1.378086e+10,342.625000,342.625000,4.302340,9.237281e+08,174.335584,174.335584,3.633679
25262,Lurie_000008,2023-01-01,38.0,2620,M,2023-01-11,67.60,349.52,417.12,134.142857,...,178.41335,3.654743,2.164830e+10,383.580000,383.580000,4.425517,9.722692e+08,176.582083,176.582083,3.645328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12567,Lurie_001800,2024-01-01,23.0,700,M,2024-04-08,337.58,21.27,358.85,141.833333,...,NaN,NaN,4.026361e+09,251.900000,251.900000,3.983887,1.488210e+08,110.450079,110.450079,3.241839
15298,Lurie_001801,2025-01-01,39.0,3665,M,2025-01-02,0.00,82.35,82.35,90.333333,...,NaN,NaN,1.677722e+07,64.000000,64.000000,2.828427,NaN,NaN,NaN,NaN
11961,Lurie_001802,2025-01-01,35.0,3970,F,2025-01-14,94.68,0.00,94.68,160.000000,...,NaN,NaN,7.059118e+08,163.000000,163.000000,3.573114,NaN,NaN,NaN,NaN
11962,Lurie_001802,2025-01-01,35.0,3970,F,2025-01-15,407.13,0.00,407.13,166.333333,...,NaN,NaN,3.023928e+09,234.500000,234.500000,3.913233,1.045404e+08,101.116270,101.116270,3.171066


## Train and Test Data Setup

Loading the consolidated csv file and dropping any missing values in `WeeklyWeightGainGPerKGPerDay`.

In [7]:
df = pd.read_csv(os.path.join(DATA_ROOT, "Consolidated.csv"))

df = df.dropna(subset=["WeeklyWeightGainGPerKGPerDay"])

1. We drop all values that are derivatives of `WeeklyWeightGainGPerKGPerDay` to avoid data leakage. Data is then split into X and y.
2. For each categorical/text column:
    - Missing values are filled with "Missing" (as a string label).
    - Each unique value is replaced with an integer (e.g., "A", "B", "Missing" → 0, 1, 2).
3. For each numeric feature, any missing values are filled in with the median value of that column.
4. All values of X are limited to between $-1000000$ and $1000000$. This is to stop massive values from overflowing.
5. Splits the data into 80% train, 20% test. A seed is set to ensure the split stays constant.

In [8]:
X = df.drop(
    columns=[
        "WeeklyWeightGainGPerDay",
        "WeeklyWeightGainGPerKGPerDay",
        "DailyWeightGainGPerDay",
        "DailyWeightGainGPerKGPerDay",
    ]
)
y = df["WeeklyWeightGainGPerKGPerDay"]

for col in X.select_dtypes(include=["object", "category"]).columns:
    X[col] = X[col].fillna("Missing")
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])

for col in X.select_dtypes(include=["float64", "int64"]).columns:
    X[col] = X[col].fillna(X[col].median())

X = X.clip(lower=-1e6, upper=1e6)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

A helper function that returns Mean Absolute Error, Mean Squared Error, and $R^2$ value for a given prediction and ground truth vector.

In [9]:
def model_results(preds, y_test):
    mae = mean_absolute_error(y_test, preds)
    mse = mean_squared_error(y_test, preds)
    r2 = r2_score(y_test, preds)

    return f"MAE: {mae} | MSE: {mse} | r^2: {r2}"

An xgboost regressor model is trained using a random search with 3 fold cross validation. Note that the commented out blcok uses a Grid Search instead.

In [10]:
param_grid: dict[str, Union[list[int], list[float]]] = {
    "n_estimators": [100, 300, 500],
    "max_depth": [3, 6, 9],
    "learning_rate": [0.01, 0.1, 0.3],
    "subsample": [0.7, 0.9, 1.0],
    "colsample_bytree": [0.7, 0.9, 1.0],
    "gamma": [0, 1, 5],
}

model = xg.XGBRegressor(objective="reg:squarederror", seed=42)
"""
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=3,
    scoring="neg_mean_squared_error",
    verbose=2,
    n_jobs=1,
)
grid_search.fit(X_train, y_train)

print("Best parameters:", grid_search.best_params_)
print("Best CV score (negative MSE):", grid_search.best_score_)

y_pred = grid_search.predict(X_test)
print(model_results(preds=y_pred, y_test=y_test))
"""

random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_grid,
    n_iter=128,
    cv=3,
    scoring="neg_mean_squared_error",
    verbose=2,
    n_jobs=1,
)
random_search.fit(X_train, y_train)

print("Best parameters:", random_search.best_params_)
print("Best CV score (negative MSE):", random_search.best_score_)

y_pred = random_search.predict(X_test)
print(model_results(preds=y_pred, y_test=y_test))

Fitting 3 folds for each of 128 candidates, totalling 384 fits
[CV] END colsample_bytree=0.9, gamma=5, learning_rate=0.3, max_depth=9, n_estimators=500, subsample=1.0; total time=   2.2s
[CV] END colsample_bytree=0.9, gamma=5, learning_rate=0.3, max_depth=9, n_estimators=500, subsample=1.0; total time=   3.0s
[CV] END colsample_bytree=0.9, gamma=5, learning_rate=0.3, max_depth=9, n_estimators=500, subsample=1.0; total time=   3.0s
[CV] END colsample_bytree=1.0, gamma=0, learning_rate=0.1, max_depth=3, n_estimators=100, subsample=1.0; total time=   0.2s
[CV] END colsample_bytree=1.0, gamma=0, learning_rate=0.1, max_depth=3, n_estimators=100, subsample=1.0; total time=   0.3s
[CV] END colsample_bytree=1.0, gamma=0, learning_rate=0.1, max_depth=3, n_estimators=100, subsample=1.0; total time=   1.2s


KeyboardInterrupt: 

Listing feature importances for the trained random search model. This is used to ID features with the strongest impact.

In [ ]:
importances = pd.Series(
    random_search.best_estimator_.feature_importances_, index=X_train.columns
).sort_values(ascending=False)
importances.head(29)

I tried to replicate the graph shown from your presentation, and as shown below, I was unsuccessful.

In [ ]:
df = X_test.copy()
df["WeeklyWeightGainPerKgPerDay"] = y_test
df["GA * 7 + DayFromBirthNumeric"] = df["GA"] * 7 + df["DayFromBirthNumeric"]

n_patients = 128
patient_ids = df["ID"].unique()[:n_patients]
df_subset = df[df["ID"].isin(patient_ids)]

fig, ax = plt.subplots(figsize=(12, 7))
color_cycle = plt.cm.tab20(np.linspace(0, 1, n_patients))
patient_colors = {pid: color_cycle[i] for i, pid in enumerate(patient_ids)}


for patient_id, group in df_subset.groupby("ID"):
    color = patient_colors[patient_id]

    x_vals = group["GA * 7 + DayFromBirthNumeric"].values
    y_real = group["WeeklyWeightGainPerKgPerDay"].values

    model_X = X_test.loc[group.index]
    y_pred = random_search.predict(model_X)

    ax.scatter(x_vals, y_real, s=22, alpha=0.7, color=color)

    for i in range(len(x_vals)):
        x_head = x_vals[i]
        y_head = y_real[i]
        pred_gain = y_pred[i]

        x_tail = x_head + 7
        y_tail = y_head + pred_gain * 7

        ax.plot([x_head, x_tail], [y_head, y_tail], color=color, linewidth=1.7)

ax.set_xlabel("GA * 7 + DayFromBirthNumeric", fontsize=14)
ax.set_ylabel("WeeklyWeightGainGPerKGPerDay", fontsize=14)
ax.set_title(
    f"First {n_patients} patients: real values (dots) & model weekly vector (lines)",
    fontsize=15,
)
plt.tight_layout()
plt.show()


## Training using Optuna
This code block uses Optuna, an optimization library to train the XGB model. The model is tested 1000 times with different hyperparameters, and the best model is chosen at the end. The process can be stopped at any time, and the best model will be saved.

In [ ]:
def objective_func(trial: optuna.Trial):
    n_estimators = trial.suggest_int("n_estimators", 100, 1000)
    max_depth = trial.suggest_int("max_depth", 10, 50)
    min_child_weight = trial.suggest_int("min_child_weight", 1, 32)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0, 1)
    objective = trial.suggest_categorical(
        "objective",
        [
            "reg:squarederror",
            "reg:absoluteerror",
        ],
    )
    learning_rate = trial.suggest_float("learning_rate", 0.005, 0.25)

    model = xg.XGBRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_child_weight=min_child_weight,
        colsample_bytree=colsample_bytree,
        objective=objective,
        learning_rate=learning_rate,
        random_state=42,
    )

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    score = r2_score(y_test, y_pred)

    return score


study = optuna.create_study(
    direction="maximize", sampler=optuna.samplers.RandomSampler(seed=42)
)
study.optimize(objective_func, n_trials=1000)

print("Best Trial")
trial = study.best_trial

print(f"Value: {trial.value}")

print("Params:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

This block prints the statistics for the best model found using Optuna.

In [ ]:
best_params = study.best_params

best_n_estimators = best_params["n_estimators"]
best_max_depth = best_params["max_depth"]
best_min_child_weight = best_params["min_child_weight"]
best_colsample_bytree = best_params["colsample_bytree"]
best_objective = best_params["objective"]
best_learning_rate = best_params["learning_rate"]

best_model = xg.XGBRegressor(
    n_estimators=best_n_estimators,
    max_depth=best_max_depth,
    min_child_weight=best_min_child_weight,
    colsample_bytree=best_colsample_bytree,
    objective=best_objective,
    learning_rate=best_learning_rate,
)

best_model.fit(X_train, y_train)

y_pred_xgb_optuna_200_trials = best_model.predict(X_test)

model_results(y_pred_xgb_optuna_200_trials, y_test)